In [25]:
# Import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
import category_encoders as ce

# Traffic Predict
We would like to predict traffic from Waze traffic data, holiday date, VisualCrossing Weather. This project will predict traffic in Kota Bandung

In [26]:
# Get Data
df_alerts_kota_bandung = pd.read_csv(r'..\Data\aggregate_alerts_Kota Bandung.csv')
df_jams = pd.read_csv(r'..\Data\aggregate_median_jams_Kota Bandung_fixed.csv')
df_weather = pd.read_csv(r'..\Data\WeatherData.csv')
df_libur = pd.read_csv(r'..\Data\HariLibur.csv')


In [14]:
df_alerts_kota_bandung.sample(4)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
2873,5956495,2022-07-12 15:00:00.000,32.73,KOTA BANDUNG,N11 Flyover Kopo-Cibaduyut,ROAD_CLOSED,"[107.59261600000005, -6.946127500000002]",120,2022-07-12
27626,6194956,2022-08-31 04:00:00.000,32.73,KOTA BANDUNG,KH Wahid Hasyim,ROAD_CLOSED,"[107.58963349999998, -6.945608999999993]",120,2022-08-31
15001,6074705,2022-08-05 17:00:00.000,32.73,KOTA BANDUNG,Paskal Hyper Square,JAM,"[107.598704, -6.914712000000002]",30,2022-08-05
1773,5944488,2022-07-09 14:00:00.000,32.73,KOTA BANDUNG,Gerbang Tol Gede Bage,ROAD_CLOSED,"[107.69057999999978, -6.959956500000006]",120,2022-07-09


In [17]:
df_alerts_kota_bandung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30385 entries, 0 to 30384
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         30385 non-null  int64  
 1   time                       30385 non-null  object 
 2   kemendagri_kabupaten_kode  30385 non-null  float64
 3   kemendagri_kabupaten_nama  30385 non-null  object 
 4   street                     28496 non-null  object 
 5   type                       30385 non-null  object 
 6   avg_location               30385 non-null  object 
 7   total_records              30385 non-null  int64  
 8   date                       30385 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 2.1+ MB


In [18]:
df_alerts_kota_bandung.shape

(30385, 9)

In [15]:
df_jams.sample(4)

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
194969,45091,2022-08-14 16:00:00.000,32.73,KOTA BANDUNG,Ir Haji Juanda,1,797.0,77.0,15.485001,18,35976346,2022-08-14,1.0,"MULTILINESTRING ((107.620845 -6.866846, 107.62..."
53701,35834,2022-07-17 13:00:00.000,32.73,KOTA BANDUNG,Cidurian Utara,2,423.0,60.0,11.285000,2,34200654,2022-07-17,2.0,"LINESTRING (107.654461 -6.934811, 107.6545 -6...."
185377,35499,2022-08-12 19:00:00.000,32.73,KOTA BANDUNG,Sukamaju,4,468.0,386.0,3.670000,14,35838521,2022-08-12,4.0,"MULTILINESTRING ((107.59987 -6.890035, 107.599..."
297136,147258,2022-09-03 23:00:00.000,32.73,KOTA BANDUNG,Jalan Sumatra,2,555.0,127.0,9.920000,1,37157244,2022-09-03,2.0,"LINESTRING (107.613323 -6.913227, 107.61373 -6..."


In [22]:
df_jams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301995 entries, 0 to 301994
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 301995 non-null  int64  
 1   time                       301995 non-null  object 
 2   kemendagri_kabupaten_kode  301995 non-null  float64
 3   kemendagri_kabupaten_nama  301995 non-null  object 
 4   street                     299552 non-null  object 
 5   level                      301995 non-null  int64  
 6   median_length              301995 non-null  float64
 7   median_delay               301995 non-null  float64
 8   median_speed_kmh           301995 non-null  float64
 9   total_records              301995 non-null  int64  
 10  id                         301995 non-null  int64  
 11  date                       301995 non-null  object 
 12  median_level               301995 non-null  float64
 13  geometry                   30

In [21]:
df_jams.shape

(301995, 14)

In [16]:
df_weather.sample(4)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
36,Bandung,2022-08-09,29.1,17.4,22.7,29.3,17.4,22.7,16.1,68.8,...,23.8,9,10,2022-08-09T05:59:36,2022-08-09T17:50:48,0.40,"Rain, Partially cloudy",Becoming cloudy in the afternoon with late aft...,rain,"96791099999,96781599999,96781099999,remote,967..."
18,Bandung,2022-07-22,26.5,18.2,22.0,26.5,18.2,22.0,18.2,80.2,...,16.8,7,10,2022-07-22T06:02:47,2022-07-22T17:49:24,0.78,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,"96791099999,96781099999,remote,96751099999"
7,Bandung,2022-07-11,28.2,16.9,22.2,28.6,16.9,22.2,16.9,74.5,...,22.1,9,10,2022-07-11T06:02:46,2022-07-11T17:47:26,0.41,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96791099999,96781099999,remote,96751099999"
20,Bandung,2022-07-24,27.9,18.5,22.3,29.9,18.5,22.4,19.0,82.7,...,17.1,7,10,2022-07-24T06:02:38,2022-07-24T17:49:41,0.85,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96791099999,96781099999,remote,96751099999"


In [19]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              66 non-null     object 
 1   datetime          66 non-null     object 
 2   tempmax           66 non-null     float64
 3   tempmin           66 non-null     float64
 4   temp              66 non-null     float64
 5   feelslikemax      66 non-null     float64
 6   feelslikemin      66 non-null     float64
 7   feelslike         66 non-null     float64
 8   dew               66 non-null     float64
 9   humidity          66 non-null     float64
 10  precip            66 non-null     float64
 11  precipprob        66 non-null     int64  
 12  precipcover       66 non-null     float64
 13  preciptype        64 non-null     object 
 14  snow              66 non-null     int64  
 15  snowdepth         66 non-null     int64  
 16  windgust          66 non-null     float64
 17 

In [27]:
df_weather.shape

(66, 33)

In [28]:
df_libur.sample(4)

,Tanggal,Hari,Hari Libur
5,4/29/2022,Jumat,Cuti Bersama Lebaran
14,6/1/2022,Rabu,Hari Lahir Pancasila
11,5/6/2022,Jumat,Cuti Bersama Lebaran
12,5/16/2022,Senin,Hari Waisak


In [30]:
df_libur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tanggal     20 non-null     object
 1   Hari        20 non-null     object
 2   Hari Libur  20 non-null     object
dtypes: object(3)
memory usage: 608.0+ bytes


In [32]:
df_libur.shape

(20, 3)

From df_alerts we can find out what happened on a certain road on a certain date. From df_jams data we can find out how congestion occurs from level 1-5. df_libur provides holiday date. df_weather provide Bandung's weather data. We will combine all of them into a dataset

## EDA aggregate_alerts_Kota Bandung.csv

In [4]:
df_alerts_kota_bandung.sample(6)

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
17434,6097524,2022-08-10 23:00:00.000,32.73,KOTA BANDUNG,Setra Duta Raya,WEATHERHAZARD,"[107.5714779999999, -6.878313999999993]",60,2022-08-10
1144,5939852,2022-07-08 14:00:00.000,32.73,KOTA BANDUNG,Alun-alun Timur,JAM,"[107.607686, -6.921693]",4,2022-07-08
3400,5961420,2022-07-13 18:00:00.000,32.73,KOTA BANDUNG,N11 Jenderal Haji Amir Machmud,JAM,"[107.567166, -6.907446]",6,2022-07-13
4779,5977777,2022-07-16 11:00:00.000,32.73,KOTA BANDUNG,N11 Soekarno-Hatta,JAM,"[107.64922065432097, -6.942159037037033]",81,2022-07-16
10773,6031914,2022-07-27 16:00:00.000,32.73,KOTA BANDUNG,Ir Haji Juanda,JAM,"[107.61634506249996, -6.880627750000004]",32,2022-07-27
5241,5978239,2022-07-16 19:00:00.000,32.73,KOTA BANDUNG,Peta,JAM,"[107.58577620000001, -6.9271806]",5,2022-07-16


In [6]:
df_alerts_kota_bandung.isnull().sum()

id                              0
time                            0
kemendagri_kabupaten_kode       0
kemendagri_kabupaten_nama       0
street                       1889
type                            0
avg_location                    0
total_records                   0
date                            0
dtype: int64

In [7]:
# Provides data with null value in 'street' column
df_alerts_nan = df_alerts_kota_bandung[df_alerts_kota_bandung['street'].isnull()]

# Delete duplicate value in 'avg_location'
df_alerts_nan.drop_duplicates('avg_location', inplace=True)
df_alerts_nan.sample(6)

C:\Users\Fachrul Ghiffari\AppData\Local\Temp\ipykernel_1088\2680646944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alerts_nan.drop_duplicates('avg_location', inplace=True)


,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date
24811,6170190,2022-08-26 11:00:00.000,32.73,KOTA BANDUNG,NaN,ROAD_CLOSED,"[107.62511883333325, -6.950247499999998]",360,2022-08-26
17386,6097475,2022-08-10 18:00:00.000,32.73,KOTA BANDUNG,NaN,JAM,"[107.60750899999995, -6.942425499999994]",60,2022-08-10
20381,6126131,2022-08-16 18:00:00.000,32.73,KOTA BANDUNG,NaN,JAM,"[107.56825335294118, -6.9301622941176495]",17,2022-08-16
5670,5984727,2022-07-17 15:00:00.000,32.73,KOTA BANDUNG,NaN,ROAD_CLOSED,"[107.65912933333331, -6.954574999999988]",165,2022-07-17
16159,6084814,2022-08-07 20:00:00.000,32.73,KOTA BANDUNG,NaN,ACCIDENT,"[107.67409600000006, -6.936128999999996]",32,2022-08-07
2808,5956433,2022-07-12 12:00:00.000,32.73,KOTA BANDUNG,NaN,ROAD_CLOSED,"[107.6833852307692, -6.9572892307692245]",130,2022-07-12


In [8]:
# Check if there is data match in 'avg_location' from df_alerts_nan
# on df_alerts_kota_bandung so we can match df_alerts_nan'street' from 'avg_location'
df_alerts_kota_bandung[df_alerts_kota_bandung['avg_location'].isin(df_alerts_nan['avg_location']) & df_alerts_kota_bandung['street'].notnull()]

,id,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,type,avg_location,total_records,date


There is no match data, so we assume missing value in 'street' column is unknown

In [33]:
# Drop data with missing value
df_alerts_kota_bandung.dropna(inplace=True)
df_alerts_kota_bandung.isnull().sum()

id                           0
time                         0
kemendagri_kabupaten_kode    0
kemendagri_kabupaten_nama    0
street                       0
type                         0
avg_location                 0
total_records                0
date                         0
dtype: int64

In [34]:
# Check unique value in 'type' column
df_alerts_kota_bandung.type.unique()

array(['WEATHERHAZARD', 'ROAD_CLOSED', 'JAM', 'ACCIDENT'], dtype=object)

In [11]:
# We will take the data with the value 'road_closed' because among others, road_closed is the planned cause
df_alert_fix = df_alerts_kota_bandung[df_alerts_kota_bandung['type'] == 'ROAD_CLOSED']

In [12]:
# Features that will be used are 'date', 'street' 'type'
df_alert_fix = df_alert_fix[['date', 'street', 'type']]

In [13]:
df_alert_fix['date'] = pd.to_datetime(df_alert_fix['date'])

In [14]:
df_alert_fix

,date,street,type
1,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
2,2022-07-06,Gerbang Tol Gede Bage,ROAD_CLOSED
4,2022-07-06,KH Wahid Hasyim,ROAD_CLOSED
5,2022-07-06,Leuwipanjang,ROAD_CLOSED
6,2022-07-06,N11 Flyover Kopo-Cibaduyut,ROAD_CLOSED
...,...,...,...
30378,2022-09-06,Cibaduyut Raya,ROAD_CLOSED
30379,2022-09-06,Gerbang Tol Gede Bage,ROAD_CLOSED
30380,2022-09-06,KH Wahid Hasyim,ROAD_CLOSED
30381,2022-09-06,Leuwipanjang,ROAD_CLOSED


In [15]:
df_alert_fix['street'].unique()

array(['Cibaduyut Raya', 'Gerbang Tol Gede Bage', 'KH Wahid Hasyim',
       'Leuwipanjang', 'N11 Flyover Kopo-Cibaduyut', 'Diponegoro',
       'Majapahit', 'Tamansari', 'LLRE Martadinata', 'Batu Basal',
       'Cipedes Selatan'], dtype=object)

In [16]:
# Checking there are duplicate data
df_alert_fix[df_alert_fix['street'] == "Cibaduyut Raya"]

,date,street,type
1,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
11,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
18,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
25,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
38,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
...,...,...,...
30335,2022-09-04,Cibaduyut Raya,ROAD_CLOSED
30347,2022-09-04,Cibaduyut Raya,ROAD_CLOSED
30361,2022-09-04,Cibaduyut Raya,ROAD_CLOSED
30370,2022-09-04,Cibaduyut Raya,ROAD_CLOSED


In [17]:
# Drop duplicate data
df_alert_fix.drop_duplicates(subset=['date', 'street'], inplace=True)

In [18]:
df_alert_fix[df_alert_fix['street'] == "Cibaduyut Raya"]

,date,street,type
1,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
510,2022-07-07,Cibaduyut Raya,ROAD_CLOSED
961,2022-07-08,Cibaduyut Raya,ROAD_CLOSED
1563,2022-07-09,Cibaduyut Raya,ROAD_CLOSED
2011,2022-07-10,Cibaduyut Raya,ROAD_CLOSED
...,...,...,...
28068,2022-09-01,Cibaduyut Raya,ROAD_CLOSED
28574,2022-09-02,Cibaduyut Raya,ROAD_CLOSED
29123,2022-09-03,Cibaduyut Raya,ROAD_CLOSED
29822,2022-09-04,Cibaduyut Raya,ROAD_CLOSED


In [19]:
df_alert_fix['street'].unique()

array(['Cibaduyut Raya', 'Gerbang Tol Gede Bage', 'KH Wahid Hasyim',
       'Leuwipanjang', 'N11 Flyover Kopo-Cibaduyut', 'Diponegoro',
       'Majapahit', 'Tamansari', 'LLRE Martadinata', 'Batu Basal',
       'Cipedes Selatan'], dtype=object)

In [20]:
df_alert_fix.shape

(320, 3)

## EDA aggregate_median_jams_Kota Bandung.csv

In [25]:
# Check missing value
df_jams.isnull().sum()

Unnamed: 0                      0
time                            0
kemendagri_kabupaten_kode       0
kemendagri_kabupaten_nama       0
street                       2443
level                           0
median_length                   0
median_delay                    0
median_speed_kmh                0
total_records                   0
id                              0
date                            0
median_level                    0
geometry                        0
dtype: int64

In [26]:
df_jams_null = df_jams[df_jams['street'].isnull()]

In [27]:
df_jams_null.sample(4)

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
107075,89208,2022-07-27 15:00:00.000,32.73,KOTA BANDUNG,NaN,4,274.0,80.5,7.56,8,34827428,2022-07-27,4.0,"MULTILINESTRING ((107.709542 -6.946285, 107.70..."
58314,40447,2022-07-18 10:00:00.000,32.73,KOTA BANDUNG,NaN,2,653.0,228.0,8.87,1,34266944,2022-07-18,2.0,"LINESTRING (107.559574 -6.935874, 107.559567 -..."
249973,100095,2022-08-26 08:00:00.000,32.73,KOTA BANDUNG,NaN,2,179.0,197.0,4.07,2,36641349,2022-08-26,2.0,"LINESTRING (107.663568 -6.934733, 107.663643 -..."
262385,112507,2022-08-28 10:00:00.000,32.73,KOTA BANDUNG,NaN,2,429.0,83.0,11.29,1,36790899,2022-08-28,2.0,"LINESTRING (107.643279 -6.916035, 107.643261 -..."


In [28]:
# Check if there is match 'street' data from 'geometry' column
df_jams[df_jams['geometry'].isin(df_jams_null['geometry']) & 
                            df_jams['street'].notnull()]

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry


In [29]:
# Since there are no data match, so we can assume these data is unknown
# So we can drop data with missing value
df_jams = df_jams.dropna()

In [30]:
df_jams.sample(4)

,Unnamed: 0,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry
149754,131887,2022-08-04 18:00:00.000,32.73,KOTA BANDUNG,Kopo,4,911.0,419.5,4.585,62,35335645,2022-08-04,4.0,"MULTILINESTRING ((107.579011 -6.961374, 107.57..."
276254,126376,2022-08-31 07:00:00.000,32.73,KOTA BANDUNG,Soekarno-Hatta (Jalur Lambat),1,1810.0,77.0,30.580,5,36959065,2022-08-31,1.0,"MULTILINESTRING ((107.642595 -6.945055, 107.64..."
133326,115459,2022-08-01 15:00:00.000,32.73,KOTA BANDUNG,Cigadung Raya Selatan,3,317.0,121.0,6.750,6,35159442,2022-08-01,3.0,"MULTILINESTRING ((107.627831 -6.888855, 107.62..."
129648,111781,2022-07-31 19:00:00.000,32.73,KOTA BANDUNG,Cihampelas,4,1266.0,367.5,8.910,6,35102854,2022-07-31,4.0,"MULTILINESTRING ((107.604218 -6.892525, 107.60..."


In [31]:
# Features that we want to use is 'date', 'street', 'level'
df_jams_fix = df_jams[['date', 'street', 'level']]
df_jams_fix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301995 entries, 0 to 301994
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    301995 non-null  object
 1   street  299552 non-null  object
 2   level   301995 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.9+ MB


In [32]:
# Since in a day there are many data for a street with many different level
# We will grouping them by 'date' and 'street' and aggregate 'level' by mode
df_jams_fix = df_jams_fix.groupby(['date', 'street']).agg(pd.Series.mode)
df_jams_fix = df_jams_fix.reset_index()

In [33]:
df_jams_fix

,date,street,level
0,2022-07-06,Abdul Rahman Saleh,2
1,2022-07-06,Akses Tol Pasteur,3
2,2022-07-06,Alkateri,2
3,2022-07-06,Alun-alun Timur,3
4,2022-07-06,Anggrek,4
...,...,...,...
22651,2022-09-06,Leuwipanjang,5
22652,2022-09-06,N11 Flyover Kopo-Cibaduyut,5
22653,2022-09-06,Prof Dr Ir Sutami,3
22654,2022-09-06,Prof Mochtar Kusumaatmadja,"[1, 2, 3]"


In [34]:
df_jams_fix['level'] = df_jams_fix['level'].astype(str)
df_jams_fix['level'] = df_jams_fix['level'].str.replace(']', '')
df_jams_fix['level'] = df_jams_fix['level'].str[-1:]
df_jams_fix['level'] = df_jams_fix['level'].astype(int)

C:\Users\Fachrul Ghiffari\AppData\Local\Temp\ipykernel_1088\2287596716.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_jams_fix['level'] = df_jams_fix['level'].str.replace(']', '')


In [35]:
df_jams_fix

,date,street,level
0,2022-07-06,Abdul Rahman Saleh,2
1,2022-07-06,Akses Tol Pasteur,3
2,2022-07-06,Alkateri,2
3,2022-07-06,Alun-alun Timur,3
4,2022-07-06,Anggrek,4
...,...,...,...
22651,2022-09-06,Leuwipanjang,5
22652,2022-09-06,N11 Flyover Kopo-Cibaduyut,5
22653,2022-09-06,Prof Dr Ir Sutami,3
22654,2022-09-06,Prof Mochtar Kusumaatmadja,3


In [36]:
df_jams_fix['date'] = pd.to_datetime(df_jams_fix['date'])
df_jams_fix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22656 entries, 0 to 22655
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    22656 non-null  datetime64[ns]
 1   street  22656 non-null  object        
 2   level   22656 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 442.6+ KB


In [37]:
street_counts = df_jams_fix.groupby('date')['street'].nunique()

In [38]:
street_counts

date
2022-07-06    367
2022-07-07    381
2022-07-08    409
2022-07-09    360
2022-07-10     14
             ... 
2022-09-01    350
2022-09-02    403
2022-09-03    420
2022-09-04    390
2022-09-06     11
Name: street, Length: 62, dtype: int64

In [39]:
# There are no level 0 data in our dataset, we should add them so in our machine learning
# There are data with level 0 
all_dates = pd.date_range(start=df_jams_fix['date'].min(), end=df_jams_fix['date'].max(), freq='D')
all_streets = df_jams_fix['street'].unique()
new_df = pd.DataFrame({'date': all_dates.repeat(len(all_streets)),
                       'street': all_streets.tolist() * len(all_dates)})
new_df['date'] = pd.to_datetime(new_df['date'])
new_df['level'] = 0

In [40]:
new_df

,date,street,level
0,2022-07-06,Abdul Rahman Saleh,0
1,2022-07-06,Akses Tol Pasteur,0
2,2022-07-06,Alkateri,0
3,2022-07-06,Alun-alun Timur,0
4,2022-07-06,Anggrek,0
...,...,...,...
73138,2022-09-06,Kasturi 1,0
73139,2022-09-06,Kencana Puri 4,0
73140,2022-09-06,Leuwi Anyar,0
73141,2022-09-06,Titiran,0


In [41]:
new_df.groupby('date')['street'].nunique()

date
2022-07-06    1161
2022-07-07    1161
2022-07-08    1161
2022-07-09    1161
2022-07-10    1161
              ... 
2022-09-02    1161
2022-09-03    1161
2022-09-04    1161
2022-09-05    1161
2022-09-06    1161
Name: street, Length: 63, dtype: int64

In [42]:
# Merge data from level 0 data and df_jams
# If there is data in certain date on certain street from df_jams
# we will use them, but if there is no data we will use level 0 data
df_jams_fix = pd.concat([df_jams_fix, new_df], ignore_index=True).drop_duplicates(subset=['date', 'street'], keep='first')

In [43]:
df_jams_fix

,date,street,level
0,2022-07-06,Abdul Rahman Saleh,2
1,2022-07-06,Akses Tol Pasteur,3
2,2022-07-06,Alkateri,2
3,2022-07-06,Alun-alun Timur,3
4,2022-07-06,Anggrek,4
...,...,...,...
95794,2022-09-06,Kasturi 1,0
95795,2022-09-06,Kencana Puri 4,0
95796,2022-09-06,Leuwi Anyar,0
95797,2022-09-06,Titiran,0


## EDA Weather

In [45]:
df_weather.sample(6)

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
49,Bandung,2022-08-22,26.7,18.5,22.3,26.7,18.5,22.3,18.5,80.4,...,18.8,7,10,2022-08-22T05:54:59,2022-08-22T17:50:15,0.83,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"96791099999,96781099999,remote,96751099999"
14,Bandung,2022-07-18,27.9,17.8,22.5,29.0,17.8,22.6,18.7,80.8,...,21.2,9,30,2022-07-18T06:02:57,2022-07-18T17:48:46,0.65,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96791099999,96781099999,remote,96751099999"
65,Bandung,2022-09-07,28.2,18.5,22.3,28.4,18.5,22.4,18.8,82.1,...,19.6,9,10,2022-09-07T05:47:22,2022-09-07T17:48:12,0.39,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96791099999,96781099999,remote,96751099999"
7,Bandung,2022-07-11,28.2,16.9,22.2,28.6,16.9,22.2,16.9,74.5,...,22.1,9,10,2022-07-11T06:02:46,2022-07-11T17:47:26,0.41,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96791099999,96781099999,remote,96751099999"
42,Bandung,2022-08-15,28.8,19.1,21.9,30.5,19.1,22.0,19.2,85.5,...,13.6,8,30,2022-08-15T05:57:41,2022-08-15T17:50:42,0.60,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"96791099999,96781099999,remote,96751099999"
33,Bandung,2022-08-06,30.1,17.9,22.9,30.5,17.9,22.8,16.7,71.6,...,23.4,9,10,2022-08-06T06:00:24,2022-08-06T17:50:44,0.30,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"96791099999,96781099999,remote,96751099999"


In [47]:
# Features data from this data we'll use are 'datetime' and 'precip'
df_weather=df_weather[['datetime', 'precip']]

In [48]:
df_weather.sample(6)

,datetime,precip
22,2022-07-26,0.200
36,2022-08-09,29.039
18,2022-07-22,0.796
9,2022-07-13,6.117
56,2022-08-29,2.418
64,2022-09-06,2.148


In [49]:
# Convert 'datetime' column type to datetime
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])

In [50]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  66 non-null     datetime64[ns]
 1   precip    66 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.2 KB


In [51]:
# Get data from July 6 - September 6 2022
# match with the date from df_jams and df_alert
start_date = pd.to_datetime('2022-07-06')
end_date = pd.to_datetime('2022-09-06')
df_weather = df_weather[(df_weather['datetime'] >= start_date) & (df_weather['datetime'] <= end_date)]

In [52]:
df_weather.rename(columns={'datetime':'date'}, inplace=True)

In [53]:
df_weather

,date,precip
2,2022-07-06,1.134
3,2022-07-07,0.018
4,2022-07-08,0.200
5,2022-07-09,0.300
6,2022-07-10,0.700
...,...,...
60,2022-09-02,0.006
61,2022-09-03,19.992
62,2022-09-04,2.999
63,2022-09-05,53.706


## EDA Holiday data

In [55]:
df_libur

,Tanggal,Hari,Hari Libur
0,1/1/2022,Sabtu,Tahun Baru Masehi
1,2/1/2022,Selasa,Tahun Baru Imlek
2,2/28/2022,Senin,Isra Mi'raj
3,3/3/2022,Kamis,Hari Suci Nyepi
4,4/15/2022,Jumat,Jumat Agung
5,4/29/2022,Jumat,Cuti Bersama Lebaran
6,5/1/2022,Minggu,Hari Buruh
7,5/2/2022,Senin,Hari Raya Idul Fitri
8,5/3/2022,Selasa,Cuti Bersama Lebaran
9,5/4/2022,Rabu,Cuti Bersama Lebaran


In [56]:
# We'll use holiday data from July 6 - September 6 2022

df_libur['Tanggal'] = pd.to_datetime(df_libur['Tanggal'])


start_date = pd.to_datetime('2022-07-06')
end_date = pd.to_datetime('2022-09-06')
df_libur = df_libur[(df_libur['Tanggal']>= start_date) & (df_libur['Tanggal'] <= end_date)]

In [57]:
df_libur

,Tanggal,Hari,Hari Libur
15,2022-07-10,Minggu,Idul Adha
16,2022-07-30,Sabtu,Tahun Baru Islam
17,2022-08-17,Rabu,Hari Kemerdekaan


In [58]:
# Change value to 1 for the holiday
df_libur['Libur'] = 1
df_libur = df_libur[['Tanggal','Libur']]

C:\Users\Fachrul Ghiffari\AppData\Local\Temp\ipykernel_1088\1790690500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_libur['Libur'] = 1


In [59]:
df_libur.rename(columns={'Tanggal':'date'}, inplace=True)
df_libur

C:\Users\Fachrul Ghiffari\AppData\Local\Temp\ipykernel_1088\462405286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_libur.rename(columns={'Tanggal':'date'}, inplace=True)


,date,Libur
15,2022-07-10,1
16,2022-07-30,1
17,2022-08-17,1


In [60]:
# We make data from July 6 - September 6 2022
# with 0 value in 'Libur' if there aren't holiday
all_dates = pd.date_range(start=df_jams_fix['date'].min(), end=df_jams_fix['date'].max(), freq='D')

In [61]:
df_libur1 = pd.DataFrame({'date': all_dates})

In [62]:
df_libur1['Libur'] = 0

In [63]:
df_libur1

,date,Libur
0,2022-07-06,0
1,2022-07-07,0
2,2022-07-08,0
3,2022-07-09,0
4,2022-07-10,0
...,...,...
58,2022-09-02,0
59,2022-09-03,0
60,2022-09-04,0
61,2022-09-05,0


In [64]:
# Merge data
df_libur = pd.concat([df_libur, df_libur1], ignore_index=True).drop_duplicates(subset='date', keep='first')

In [65]:
df_libur

,date,Libur
0,2022-07-10,1
1,2022-07-30,1
2,2022-08-17,1
3,2022-07-06,0
4,2022-07-07,0
...,...,...
61,2022-09-02,0
62,2022-09-03,0
63,2022-09-04,0
64,2022-09-05,0


## Merge All Data

In [66]:
df_alert_fix

,date,street,type
1,2022-07-06,Cibaduyut Raya,ROAD_CLOSED
2,2022-07-06,Gerbang Tol Gede Bage,ROAD_CLOSED
4,2022-07-06,KH Wahid Hasyim,ROAD_CLOSED
5,2022-07-06,Leuwipanjang,ROAD_CLOSED
6,2022-07-06,N11 Flyover Kopo-Cibaduyut,ROAD_CLOSED
...,...,...,...
30378,2022-09-06,Cibaduyut Raya,ROAD_CLOSED
30379,2022-09-06,Gerbang Tol Gede Bage,ROAD_CLOSED
30380,2022-09-06,KH Wahid Hasyim,ROAD_CLOSED
30381,2022-09-06,Leuwipanjang,ROAD_CLOSED


In [67]:
df_jams_fix

,date,street,level
0,2022-07-06,Abdul Rahman Saleh,2
1,2022-07-06,Akses Tol Pasteur,3
2,2022-07-06,Alkateri,2
3,2022-07-06,Alun-alun Timur,3
4,2022-07-06,Anggrek,4
...,...,...,...
95794,2022-09-06,Kasturi 1,0
95795,2022-09-06,Kencana Puri 4,0
95796,2022-09-06,Leuwi Anyar,0
95797,2022-09-06,Titiran,0


In [68]:
df_weather

,date,precip
2,2022-07-06,1.134
3,2022-07-07,0.018
4,2022-07-08,0.200
5,2022-07-09,0.300
6,2022-07-10,0.700
...,...,...
60,2022-09-02,0.006
61,2022-09-03,19.992
62,2022-09-04,2.999
63,2022-09-05,53.706


In [69]:
df_libur

,date,Libur
0,2022-07-10,1
1,2022-07-30,1
2,2022-08-17,1
3,2022-07-06,0
4,2022-07-07,0
...,...,...
61,2022-09-02,0
62,2022-09-03,0
63,2022-09-04,0
64,2022-09-05,0


In [70]:
df_merge = pd.merge(df_jams_fix, df_weather, how='left', on='date')

In [71]:
df_merge

,date,street,level,precip
0,2022-07-06,Abdul Rahman Saleh,2,1.134
1,2022-07-06,Akses Tol Pasteur,3,1.134
2,2022-07-06,Alkateri,2,1.134
3,2022-07-06,Alun-alun Timur,3,1.134
4,2022-07-06,Anggrek,4,1.134
...,...,...,...,...
73138,2022-09-06,Kasturi 1,0,2.148
73139,2022-09-06,Kencana Puri 4,0,2.148
73140,2022-09-06,Leuwi Anyar,0,2.148
73141,2022-09-06,Titiran,0,2.148


In [72]:
df_merge = pd.merge(df_merge, df_libur, on='date', how='left')

In [73]:
df_merge

,date,street,level,precip,Libur
0,2022-07-06,Abdul Rahman Saleh,2,1.134,0
1,2022-07-06,Akses Tol Pasteur,3,1.134,0
2,2022-07-06,Alkateri,2,1.134,0
3,2022-07-06,Alun-alun Timur,3,1.134,0
4,2022-07-06,Anggrek,4,1.134,0
...,...,...,...,...,...
73138,2022-09-06,Kasturi 1,0,2.148,0
73139,2022-09-06,Kencana Puri 4,0,2.148,0
73140,2022-09-06,Leuwi Anyar,0,2.148,0
73141,2022-09-06,Titiran,0,2.148,0


In [74]:
df_merge = pd.merge(df_merge, df_alert_fix, on=['date','street'], how='left')

In [75]:
df_merge

,date,street,level,precip,Libur,type
0,2022-07-06,Abdul Rahman Saleh,2,1.134,0,NaN
1,2022-07-06,Akses Tol Pasteur,3,1.134,0,NaN
2,2022-07-06,Alkateri,2,1.134,0,NaN
3,2022-07-06,Alun-alun Timur,3,1.134,0,NaN
4,2022-07-06,Anggrek,4,1.134,0,NaN
...,...,...,...,...,...,...
73138,2022-09-06,Kasturi 1,0,2.148,0,NaN
73139,2022-09-06,Kencana Puri 4,0,2.148,0,NaN
73140,2022-09-06,Leuwi Anyar,0,2.148,0,NaN
73141,2022-09-06,Titiran,0,2.148,0,NaN


In [76]:
df_merge['day'] = df_merge['date'].dt.day_name()

In [77]:
df_merge

,date,street,level,precip,Libur,type,day
0,2022-07-06,Abdul Rahman Saleh,2,1.134,0,NaN,Wednesday
1,2022-07-06,Akses Tol Pasteur,3,1.134,0,NaN,Wednesday
2,2022-07-06,Alkateri,2,1.134,0,NaN,Wednesday
3,2022-07-06,Alun-alun Timur,3,1.134,0,NaN,Wednesday
4,2022-07-06,Anggrek,4,1.134,0,NaN,Wednesday
...,...,...,...,...,...,...,...
73138,2022-09-06,Kasturi 1,0,2.148,0,NaN,Tuesday
73139,2022-09-06,Kencana Puri 4,0,2.148,0,NaN,Tuesday
73140,2022-09-06,Leuwi Anyar,0,2.148,0,NaN,Tuesday
73141,2022-09-06,Titiran,0,2.148,0,NaN,Tuesday


In [78]:
df_merge.drop('date', axis=1, inplace=True)

In [79]:
df_merge

,street,level,precip,Libur,type,day
0,Abdul Rahman Saleh,2,1.134,0,NaN,Wednesday
1,Akses Tol Pasteur,3,1.134,0,NaN,Wednesday
2,Alkateri,2,1.134,0,NaN,Wednesday
3,Alun-alun Timur,3,1.134,0,NaN,Wednesday
4,Anggrek,4,1.134,0,NaN,Wednesday
...,...,...,...,...,...,...
73138,Kasturi 1,0,2.148,0,NaN,Tuesday
73139,Kencana Puri 4,0,2.148,0,NaN,Tuesday
73140,Leuwi Anyar,0,2.148,0,NaN,Tuesday
73141,Titiran,0,2.148,0,NaN,Tuesday


In [80]:
# Encoding 'type' column
pd.get_dummies(df_merge['type'])

,ROAD_CLOSED
0,0
1,0
2,0
3,0
4,0
...,...
73138,0
73139,0
73140,0
73141,0


In [81]:
df_merge = df_merge.join(pd.get_dummies(df_merge['type']))

In [82]:
df_merge.drop('type', axis=1, inplace=True)

In [83]:
df_merge

,street,level,precip,Libur,day,ROAD_CLOSED
0,Abdul Rahman Saleh,2,1.134,0,Wednesday,0
1,Akses Tol Pasteur,3,1.134,0,Wednesday,0
2,Alkateri,2,1.134,0,Wednesday,0
3,Alun-alun Timur,3,1.134,0,Wednesday,0
4,Anggrek,4,1.134,0,Wednesday,0
...,...,...,...,...,...,...
73138,Kasturi 1,0,2.148,0,Tuesday,0
73139,Kencana Puri 4,0,2.148,0,Tuesday,0
73140,Leuwi Anyar,0,2.148,0,Tuesday,0
73141,Titiran,0,2.148,0,Tuesday,0


In [85]:
# Merge all data
df_merge.to_csv('..\Hasil Proses Data\Data Preparation.csv', index=False)